## GAN

In [ ]:
%matplotlib inline
from fastai.gen_doc.nbdoc import *
from fastai.vision import * 
from fastai.vision.gan import *

GAN significa [Generative Adversarial Nets](https://arxiv.org/pdf/1406.2661.pdf) e foram inventadas por Ian Goodfellow. O conceito é que vamos treinar dois modelos ao mesmo tempo: um gerador e um crítico. O gerador vai tentar fazer novas imagens semelhantes às de nosso conjunto de dados, e o trabalho do crítico vai tentar classificar imagens reais dos falsos o gerador faz. O gerador de imagens, retorna o discriminadora um mapa de características (que pode ser um único número consoante o tamanho da entrada). Normalmente, o discriminador serão treinados para retornar 0. toda parte por imagens falsas e 1. todos os lugares para os reais.
Este módulo contém todas as funções necessárias para criar uma GAN.

Nós treiná-los uns contra os outros no sentido de que em cada etapa (mais ou menos), temos:
1. Congelar o gerador e treinar o discriminador de um passo por:
- recebendo um lote de imagens reais (vamos chamá que `real`)
- gerando um lote de imagens falsas (vamos chamá que `fake`)
- tem o discriminador avaliar cada lote e calcular uma função de perda daquele; a parte mais importante é que ele recompensa positivamente a detecção de imagens reais e penaliza os falsos
- actualizar os pesos do discriminadora com os gradientes de esta perda


2. Congelar o discriminador e treinar o gerador para um passo por:
- gerando um lote de imagens falsas
- avaliar o discriminador sobre ele
- devolver uma perda que premia posisitivly o discriminador pensando essas são imagens reais; a parte mais importante é que ele recompensa positivamente a detecção de imagens reais e penaliza os falsos
- actualizar os pesos do gerador com os gradientes de esta perda

In [ ]:
show_doc(GANLearner)

<h2 id="GANLearner" class="doc_header"><code>class</code> <code>GANLearner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L209" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANLearner-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>GANLearner</code>(**`data`**:[`DataBunch`](/basic_data.html#DataBunch), **`generator`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`critic`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`gen_loss_func`**:`LossFunction`, **`crit_loss_func`**:`LossFunction`, **`switcher`**:[`Callback`](/callback.html#Callback)=***`None`***, **`gen_first`**:`bool`=***`False`***, **`switch_eval`**:`bool`=***`True`***, **`show_img`**:`bool`=***`True`***, **`clip`**:`float`=***`None`***, **\*\*`learn_kwargs`**) :: [`Learner`](/basic_train.html#Learner)

<div class="collapse" id="GANLearner-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANLearner-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>GANLearner</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

A [`Learner`](/basic_train.html#Learner) suitable for GANs.  

Este é o construtor geral para criar uma GAN, você pode querer usar um dos métodos de fábrica que são mais fáceis de usar. Criar um GAN de [`data`](/vision.data.html#vision.data), um `generator` e um` critic`. O [`data`](/vision.data.html#vision.data) deve ter as entradas do `generator` vai esperar e as imagens queria como alvos.
`Gen_loss_func` é a função de perda que vai ser aplicado ao` generator`. Leva três argumento `fake_pred`,` target`, `output` e deve retornar um ranking 0 tensor. `Output` é o resultado de as` generator` aplicados à entrada (xs do lote), `target` é os ys do lote e` fake_pred` é o resultado de a `` discriminator` sendo dada output`. `` Output`and target` pode ser usado para adicionar uma perda específica para a perda GAN (perda pixel, perda de função) e por uma boa formação do gan, a perda deve encorajar `fake_pred` ser tão próximo de 1 quanto possível (o `generator` é treinado para enganar o` critic`).
`Crit_loss_func` é a função de perda que vai ser aplicado ao` critic`. Possui dois argumentos `real_pred` e` fake_pred`. `Real_pred` é o resultado do` critic` sobre as imagens do alvo (os ys do lote) e `fake_pred` é o resultado do` critic` aplicada sobre um lote de falsificação, gerado BYT a `generator` dos xs do lote.
`Switcher` é um [`Callback`](/callback.html#Callback) que deve dizer ao GAN quando mudar de crítico ao gerador e vice-versa. Por padrão ele faz 5 iterações do crítico por 1 iteração do gerador. O modelo começa o treinamento com o `generator` se` gen_first = true`. Se `switch_eval = true`, o modelo que não é treinado está ligado modo de eval (deixado no modo de treino de outra forma, o que significa que algumas estatísticas como o funcionamento significa em camadas batchnorm são atualizados, ou as desistências são aplicadas).
`Clip` deve ser definido como um determinado valor quando se quer juntar os pesos (ver o [Wassertein GAN](https://arxiv.org/pdf/1701.07875.pdf) por exemplo).
Se `show_img = TRUE, uma imagem gerada pelo GAN é apresentado no final de cada época.

### métodos de fábrica

In [ ]:
show_doc(GANLearner.from_learners)

<h4 id="GANLearner.from_learners" class="doc_header"><code>from_learners</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L222" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANLearner-from_learners-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>from_learners</code>(**`learn_gen`**:[`Learner`](/basic_train.html#Learner), **`learn_crit`**:[`Learner`](/basic_train.html#Learner), **`switcher`**:[`Callback`](/callback.html#Callback)=***`None`***, **`weights_gen`**:`Point`=***`None`***, **\*\*`learn_kwargs`**)

<div class="collapse" id="GANLearner-from_learners-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANLearner-from_learners-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>from_learners</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create a GAN from `learn_gen` and `learn_crit`.  

Directamente cria um [`GANLearner`](/vision.gan.html#GANLearner) de dois [`Learner`](/basic_train.html#Learner): uma para a `generator` e uma para a` critic`. A `` switcher` e todos kwargs` vai ser passada para a inicialização de [`GANLearner`](/vision.gan.html#GANLearner) juntamente com as seguintes funções de perda:
- `loss_func_crit` é a média de learn_crit.loss_func`` `aplicado a real_pred` e um alvo de uns com` learn_crit.loss_func` aplicados a `fake_pred` e um alvo de zeros
- `loss_func_gen` é a média de learn_crit.loss_func`` `aplicado a fake_pred` e um alvo de uns (a cheio o discriminadora) com` learn_gen.loss_func` aplicado a `` output` e target`. Os pesos de cada uma destas contribuições pode ser passado em `weights_gen` (padrão é 1. 1. e)

In [ ]:
show_doc(GANLearner.wgan)

<h4 id="GANLearner.wgan" class="doc_header"><code>wgan</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L229" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANLearner-wgan-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>wgan</code>(**`data`**:[`DataBunch`](/basic_data.html#DataBunch), **`generator`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`critic`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`switcher`**:[`Callback`](/callback.html#Callback)=***`None`***, **`clip`**:`float`=***`0.01`***, **\*\*`learn_kwargs`**)

<div class="collapse" id="GANLearner-wgan-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANLearner-wgan-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>wgan</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create a WGAN from `data`, `generator` and `critic`.  

O Wasserstein GAN é detalhada em init [this article]. `switcher` and the `kwargs` will be passed to the [`GANLearner`](/vision.gan.html#GANLearner), `clip`is o recorte peso.

## switchers

Em qualquer formação GAN, você precisa dizer ao [`Learner`](/basic_train.html#Learner) quando mudar de gerador para o crítico e vice-versa. Os dois seguintes [`Callback`](/callback.html#Callback) são exemplos para ajudá-lo com isso.
Como de costume, não chame os `métodos on_something` diretamente, a biblioteca fastai vai fazer isso por você durante o treinamento.

In [ ]:
show_doc(FixedGANSwitcher, title_level=3)

<h3 id="FixedGANSwitcher" class="doc_header"><code>class</code> <code>FixedGANSwitcher</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L156" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#FixedGANSwitcher-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>FixedGANSwitcher</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`n_crit`**:`Union`\[`int`, `Callable`\]=***`1`***, **`n_gen`**:`Union`\[`int`, `Callable`\]=***`1`***) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

<div class="collapse" id="FixedGANSwitcher-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#FixedGANSwitcher-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>FixedGANSwitcher</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Switcher to do `n_crit` iterations of the critic then `n_gen` iterations of the generator.  

In [ ]:
show_doc(FixedGANSwitcher.on_train_begin)

<h4 id="FixedGANSwitcher.on_train_begin" class="doc_header"><code>on_train_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L162" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#FixedGANSwitcher-on_train_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_train_begin</code>(**\*\*`kwargs`**)

<div class="collapse" id="FixedGANSwitcher-on_train_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#FixedGANSwitcher-on_train_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_train_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Initiate the iteration counts.  

In [ ]:
show_doc(FixedGANSwitcher.on_batch_end)

<h4 id="FixedGANSwitcher.on_batch_end" class="doc_header"><code>on_batch_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L166" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#FixedGANSwitcher-on_batch_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_batch_end</code>(**`iteration`**, **\*\*`kwargs`**)

<div class="collapse" id="FixedGANSwitcher-on_batch_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#FixedGANSwitcher-on_batch_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_batch_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Switch the model if necessary.  

In [ ]:
show_doc(AdaptiveGANSwitcher, title_level=3)

<h3 id="AdaptiveGANSwitcher" class="doc_header"><code>class</code> <code>AdaptiveGANSwitcher</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L180" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AdaptiveGANSwitcher-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>AdaptiveGANSwitcher</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`gen_thresh`**:`float`=***`None`***, **`critic_thresh`**:`float`=***`None`***) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

<div class="collapse" id="AdaptiveGANSwitcher-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AdaptiveGANSwitcher-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>AdaptiveGANSwitcher</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Switcher that goes back to generator/critic when the loss goes below `gen_thresh`/`crit_thresh`.  

In [ ]:
show_doc(AdaptiveGANSwitcher.on_batch_end)

<h4 id="AdaptiveGANSwitcher.on_batch_end" class="doc_header"><code>on_batch_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L186" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AdaptiveGANSwitcher-on_batch_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_batch_end</code>(**`last_loss`**, **\*\*`kwargs`**)

<div class="collapse" id="AdaptiveGANSwitcher-on_batch_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AdaptiveGANSwitcher-on_batch_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_batch_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Switch the model if necessary.  

## discriminativo LR

Se você quer treinar seu crítico a um ritmo de aprendizagem diferente do que o gerador, isso vai deixar você fazer isso automaticamente (mesmo se você tem uma agenda taxa de aprendizagem).

In [ ]:
show_doc(GANDiscriminativeLR, title_level=3)

<h3 id="GANDiscriminativeLR" class="doc_header"><code>class</code> <code>GANDiscriminativeLR</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L284" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANDiscriminativeLR-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>GANDiscriminativeLR</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`mult_lr`**:`float`=***`5.0`***) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

<div class="collapse" id="GANDiscriminativeLR-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANDiscriminativeLR-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>GANDiscriminativeLR</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

[`Callback`](/callback.html#Callback) that handles multiplying the learning rate by `mult_lr` for the critic.  

In [ ]:
show_doc(GANDiscriminativeLR.on_batch_begin)

<h4 id="GANDiscriminativeLR.on_batch_begin" class="doc_header"><code>on_batch_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L290" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANDiscriminativeLR-on_batch_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_batch_begin</code>(**`train`**, **\*\*`kwargs`**)

<div class="collapse" id="GANDiscriminativeLR-on_batch_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANDiscriminativeLR-on_batch_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_batch_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Multiply the current lr if necessary.  

In [ ]:
show_doc(GANDiscriminativeLR.on_step_end)

<h4 id="GANDiscriminativeLR.on_step_end" class="doc_header"><code>on_step_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L294" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANDiscriminativeLR-on_step_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_step_end</code>(**\*\*`kwargs`**)

<div class="collapse" id="GANDiscriminativeLR-on_step_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANDiscriminativeLR-on_step_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_step_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Put the LR back to its value if necessary.  

## modelos específicos

In [ ]:
show_doc(basic_critic)

<h4 id="basic_critic" class="doc_header"><code>basic_critic</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L17" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#basic_critic-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>basic_critic</code>(**`in_size`**:`int`, **`n_channels`**:`int`, **`n_features`**:`int`=***`64`***, **`n_extra_layers`**:`int`=***`0`***, **\*\*`conv_kwargs`**)

<div class="collapse" id="basic_critic-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#basic_critic-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>basic_critic</code>:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_basic_critic</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L55" class="source_link" style="float:right">[source]</a></li></ul><p>Some other tests where <code>basic_critic</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_module</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L66" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

A basic critic for images `n_channels` x `in_size` x `in_size`.  

Este modelo contém uma primeira camada 4 por 4 convolucional do passo 2 do `` n_channels` para n_features` seguido por `n_extra_layers` três por três camada convolucional do passo 1. Em seguida, colocar o maior número de 4 por 4 camada convolucional do passo 2 com um número de características multiplicado por 2 em cada fase de modo que a `` in_size` torna-se 1. kwargs` pode ser utilizado para personalizar as camadas convolucionais e são passados ​​para [`conv_layer`](/layers.html#conv_layer).

In [ ]:
show_doc(basic_generator)

<h4 id="basic_generator" class="doc_header"><code>basic_generator</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L28" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#basic_generator-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>basic_generator</code>(**`in_size`**:`int`, **`n_channels`**:`int`, **`noise_sz`**:`int`=***`100`***, **`n_features`**:`int`=***`64`***, **`n_extra_layers`**=***`0`***, **\*\*`conv_kwargs`**)

<div class="collapse" id="basic_generator-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#basic_generator-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>basic_generator</code>:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_basic_generator</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L45" class="source_link" style="float:right">[source]</a></li></ul><p>Some other tests where <code>basic_generator</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_module</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L66" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

A basic generator from `noise_sz` to images `n_channels` x `in_size` x `in_size`.  

Este modelo contém um primeiro quatro por quatro transposta camada convolucional do passo 1 do `noise_size` para os últimos números de características do crítico correspondente. Então nós colocamos como muitos 4 por 4 camada convolutional transposta do passo 2 com um número de características dividido por 2 em cada etapa para que a imagem acaba sendo de altura e Largura `in_size // 2`. No final, que add`n_extra_layers` três por três camada convolucional do passo 1. A última camada é uma convolução transposta de tamanho 4 por 4 e passo 2 seguido por `tanh`. `Kwargs` pode ser utilizado para personalizar as camadas convolucionais e são passados ​​para [`conv_layer`](/layers.html#conv_layer).

In [ ]:
show_doc(gan_critic)

<h4 id="gan_critic" class="doc_header"><code>gan_critic</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L267" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#gan_critic-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>gan_critic</code>(**`n_channels`**:`int`=***`3`***, **`nf`**:`int`=***`128`***, **`n_blocks`**:`int`=***`3`***, **`p`**:`int`=***`0.15`***)

<div class="collapse" id="gan_critic-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#gan_critic-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>gan_critic</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Critic to train a `GAN`.  

In [ ]:
show_doc(GANTrainer)

<h2 id="GANTrainer" class="doc_header"><code>class</code> <code>GANTrainer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L71" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>GANTrainer</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`switch_eval`**:`bool`=***`False`***, **`clip`**:`float`=***`None`***, **`beta`**:`float`=***`0.98`***, **`gen_first`**:`bool`=***`False`***, **`show_img`**:`bool`=***`True`***) :: [`LearnerCallback`](/basic_train.html#LearnerCallback)

<div class="collapse" id="GANTrainer-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>GANTrainer</code>:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_trainer</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L80" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Handles GAN Training.  

[`LearnerCallback`](/basic_train.html#LearnerCallback) que será responsável para lidar com os dois otimizadores diferentes (um para o gerador e um para o crítico), e fazer todo o trabalho nos bastidores para que o gerador (ou o crítico) estão em modo de treino com a exigência parâmetros gradientes de cada vez vamos mudar.
`Switch_eval = true` significa que o [`GANTrainer`](/vision.gan.html#GANTrainer) vai colocar o modelo que não está treinando em modo de eval (se é` false` suas estatísticas de funcionamento como em camadas batchnorm será atualizado e abandono serão aplicadas). `Clip` é o recorte aplicadas aos pesos (se não` none`). `Beta` é o coeficiente para as médias móveis como os [`GANTrainer`](/vision.gan.html#GANTrainer)tracks separadamente o gerador de perda e a perda crítico. `Gen_first = true` significa o treinamento começa com o gerador (com o crítico se é` false`). Se `show_img = true` que mostram uma imagem gerada no final de cada época.

In [ ]:
show_doc(GANTrainer.switch)

<h4 id="GANTrainer.switch" class="doc_header"><code>switch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L148" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-switch-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>switch</code>(**`gen_mode`**:`bool`=***`None`***)

<div class="collapse" id="GANTrainer-switch-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-switch-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>switch</code>:</p><p>Some other tests where <code>switch</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_module</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L66" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Switch the model, if `gen_mode` is provided, in the desired mode.  

Se `gen_mode` é deixada como` none`, basta colocar o modelo no outro modo (crítico se fosse no modo gerador e vice-versa).

In [ ]:
show_doc(GANTrainer.on_train_begin)

<h4 id="GANTrainer.on_train_begin" class="doc_header"><code>on_train_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L89" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-on_train_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_train_begin</code>(**\*\*`kwargs`**)

<div class="collapse" id="GANTrainer-on_train_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-on_train_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_train_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create the optimizers for the generator and critic if necessary, initialize smootheners.  

In [ ]:
show_doc(GANTrainer.on_epoch_begin)

<h4 id="GANTrainer.on_epoch_begin" class="doc_header"><code>on_epoch_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L131" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-on_epoch_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_epoch_begin</code>(**`epoch`**, **\*\*`kwargs`**)

<div class="collapse" id="GANTrainer-on_epoch_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-on_epoch_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_epoch_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Put the critic or the generator back to eval if necessary.  

In [ ]:
show_doc(GANTrainer.on_batch_begin)

<h4 id="GANTrainer.on_batch_begin" class="doc_header"><code>on_batch_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L109" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-on_batch_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_batch_begin</code>(**`last_input`**, **`last_target`**, **\*\*`kwargs`**)

<div class="collapse" id="GANTrainer-on_batch_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-on_batch_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_batch_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Clamp the weights with `self.clip` if it's not None, return the correct input.  

In [ ]:
show_doc(GANTrainer.on_backward_begin)

<h4 id="GANTrainer.on_backward_begin" class="doc_header"><code>on_backward_begin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L116" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-on_backward_begin-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_backward_begin</code>(**`last_loss`**, **`last_output`**, **\*\*`kwargs`**)

<div class="collapse" id="GANTrainer-on_backward_begin-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-on_backward_begin-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_backward_begin</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Record `last_loss` in the proper list.  

In [ ]:
show_doc(GANTrainer.on_epoch_end)

<h4 id="GANTrainer.on_epoch_end" class="doc_header"><code>on_epoch_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L135" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-on_epoch_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_epoch_end</code>(**`pbar`**, **`epoch`**, **`last_metrics`**, **\*\*`kwargs`**)

<div class="collapse" id="GANTrainer-on_epoch_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-on_epoch_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_epoch_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Put the various losses in the recorder and show a sample image.  

In [ ]:
show_doc(GANTrainer.on_train_end)

<h4 id="GANTrainer.on_train_end" class="doc_header"><code>on_train_end</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L105" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANTrainer-on_train_end-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_train_end</code>(**\*\*`kwargs`**)

<div class="collapse" id="GANTrainer-on_train_end-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANTrainer-on_train_end-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_train_end</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Switch in generator mode for showing results.  

## módulos específicos

In [ ]:
show_doc(GANModule, title_level=3)

<h3 id="GANModule" class="doc_header"><code>class</code> <code>GANModule</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L41" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANModule-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>GANModule</code>(**`generator`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=***`None`***, **`critic`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=***`None`***, **`gen_mode`**:`bool`=***`False`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="GANModule-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANModule-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>GANModule</code>:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_module</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L66" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Wrapper around a `generator` and a `critic` to create a GAN.  

Se `gen_mode` é deixada como` none`, basta colocar o modelo no outro modo (crítico se fosse no modo gerador e vice-versa).

In [ ]:
show_doc(GANModule.switch)

<h4 id="GANModule.switch" class="doc_header"><code>switch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L50" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANModule-switch-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>switch</code>(**`gen_mode`**:`bool`=***`None`***)

<div class="collapse" id="GANModule-switch-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANModule-switch-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>switch</code>:</p><p>Some other tests where <code>switch</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_module</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L66" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Put the model in generator mode if `gen_mode`, in critic mode otherwise.  

In [ ]:
show_doc(GANLoss, title_level=3)

<h3 id="GANLoss" class="doc_header"><code>class</code> <code>GANLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L54" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANLoss-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>GANLoss</code>(**`loss_funcG`**:`Callable`, **`loss_funcC`**:`Callable`, **`gan_model`**:[`GANModule`](/vision.gan.html#GANModule)) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`GANModule`](/vision.gan.html#GANModule)

<div class="collapse" id="GANLoss-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANLoss-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>GANLoss</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Wrapper around `loss_funcC` (for the critic) and `loss_funcG` (for the generator).  

In [ ]:
show_doc(AdaptiveLoss, title_level=3)

<h3 id="AdaptiveLoss" class="doc_header"><code>class</code> <code>AdaptiveLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L298" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AdaptiveLoss-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>AdaptiveLoss</code>(**`crit`**) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="AdaptiveLoss-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AdaptiveLoss-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>AdaptiveLoss</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Expand the `target` to match the `output` size before applying `crit`.  

In [ ]:
show_doc(accuracy_thresh_expand)

<h4 id="accuracy_thresh_expand" class="doc_header"><code>accuracy_thresh_expand</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L306" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#accuracy_thresh_expand-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>accuracy_thresh_expand</code>(**`y_pred`**:`Tensor`, **`y_true`**:`Tensor`, **`thresh`**:`float`=***`0.5`***, **`sigmoid`**:`bool`=***`True`***) → `Rank0Tensor`

<div class="collapse" id="accuracy_thresh_expand-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#accuracy_thresh_expand-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>accuracy_thresh_expand</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Compute accuracy after expanding `y_true` to the size of `y_pred`.  

## Data Block API

In [ ]:
show_doc(NoisyItem, title_level=3)

<h3 id="NoisyItem" class="doc_header"><code>class</code> <code>NoisyItem</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L234" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#NoisyItem-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>NoisyItem</code>(**`noise_sz`**) :: [`ItemBase`](/core.html#ItemBase)

<div class="collapse" id="NoisyItem-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#NoisyItem-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>NoisyItem</code>:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_noisy_item</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L37" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

An random [`ItemBase`](/core.html#ItemBase) of size `noise_sz`.  

In [ ]:
show_doc(GANItemList, title_level=3)

<h3 id="GANItemList" class="doc_header"><code>class</code> <code>GANItemList</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L242" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANItemList-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>GANItemList</code>(**`items`**, **`noise_sz`**:`int`=***`100`***, **\*\*`kwargs`**) :: [`ImageList`](/vision.data.html#ImageList)

<div class="collapse" id="GANItemList-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANItemList-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>GANItemList</code>:</p><p>Some other tests where <code>GANItemList</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_gan_datasets</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L30" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

`ItemList` suitable for GANs.  

Entradas será [`NoisyItem`](/vision.gan.html#NoisyItem) de `noise_sz` enquanto a classe padrão para meta é [`ImageList`](/vision.data.html#ImageList).

In [ ]:
show_doc(GANItemList.show_xys)

<h4 id="GANItemList.show_xys" class="doc_header"><code>show_xys</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L254" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANItemList-show_xys-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>show_xys</code>(**`xs`**, **`ys`**, **`imgsize`**:`int`=***`4`***, **`figsize`**:`Optional`\[`Tuple`\[`int`, `int`\]\]=***`None`***, **\*\*`kwargs`**)

<div class="collapse" id="GANItemList-show_xys-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANItemList-show_xys-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>show_xys</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Shows `ys` (target images) on a figure of `figsize`.  

In [ ]:
show_doc(GANItemList.show_xyzs)

<h4 id="GANItemList.show_xyzs" class="doc_header"><code>show_xyzs</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L258" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANItemList-show_xyzs-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>show_xyzs</code>(**`xs`**, **`ys`**, **`zs`**, **`imgsize`**:`int`=***`4`***, **`figsize`**:`Optional`\[`Tuple`\[`int`, `int`\]\]=***`None`***, **\*\*`kwargs`**)

<div class="collapse" id="GANItemList-show_xyzs-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANItemList-show_xyzs-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>show_xyzs</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Shows `zs` (generated images) on a figure of `figsize`.  

## Indocumentados Métodos - Métodos movidos abaixo desta linha irá intencionalmente ser escondido

In [ ]:
show_doc(GANLoss.critic)

<h4 id="GANLoss.critic" class="doc_header"><code>critic</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L65" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANLoss-critic-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>critic</code>(**`real_pred`**, **`input`**)

<div class="collapse" id="GANLoss-critic-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANLoss-critic-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>critic</code>:</p><p>Some other tests where <code>critic</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_basic_critic</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L55" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Create some `fake_pred` with the generator from `input` and compare them to `real_pred` in `self.loss_funcD`.  

In [ ]:
show_doc(GANModule.forward)

<h4 id="GANModule.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L47" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANModule-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**\*`args`**)

<div class="collapse" id="GANModule-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANModule-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(GANLoss.generator)

<h4 id="GANLoss.generator" class="doc_header"><code>generator</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L60" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANLoss-generator-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>generator</code>(**`output`**, **`target`**)

<div class="collapse" id="GANLoss-generator-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANLoss-generator-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>generator</code>:</p><p>Some other tests where <code>generator</code> is used:</p><ul><li><code>pytest -sv tests/test_vision_gan.py::test_basic_generator</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_vision_gan.py#L45" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Evaluate the `output` with the critic then uses `self.loss_funcG` to combine it with `target`.  

In [ ]:
show_doc(NoisyItem.apply_tfms)

<h4 id="NoisyItem.apply_tfms" class="doc_header"><code>apply_tfms</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L238" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#NoisyItem-apply_tfms-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>apply_tfms</code>(**`tfms`**, **\*\*`kwargs`**)

<div class="collapse" id="NoisyItem-apply_tfms-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#NoisyItem-apply_tfms-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>apply_tfms</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Subclass this method if you want to apply data augmentation with `tfms` to this [`ItemBase`](/core.html#ItemBase).  

In [ ]:
show_doc(AdaptiveLoss.forward)

<h4 id="AdaptiveLoss.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L303" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AdaptiveLoss-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`output`**, **`target`**)

<div class="collapse" id="AdaptiveLoss-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AdaptiveLoss-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(GANItemList.get)

<h4 id="GANItemList.get" class="doc_header"><code>get</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L251" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANItemList-get-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>get</code>(**`i`**)

<div class="collapse" id="GANItemList-get-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANItemList-get-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>get</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Subclass if you want to customize how to create item `i` from `self.items`.  

In [ ]:
show_doc(GANItemList.reconstruct)

<h4 id="GANItemList.reconstruct" class="doc_header"><code>reconstruct</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L252" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#GANItemList-reconstruct-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>reconstruct</code>(**`t`**)

<div class="collapse" id="GANItemList-reconstruct-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#GANItemList-reconstruct-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>reconstruct</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Reconstruct one of the underlying item for its data `t`.  

In [ ]:
show_doc(AdaptiveLoss.forward)

<h4 id="AdaptiveLoss.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/vision/gan.py#L303" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#AdaptiveLoss-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`output`**, **`target`**)

<div class="collapse" id="AdaptiveLoss-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#AdaptiveLoss-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

## Novos Métodos - Por favor, documento ou mover para a seção em situação irregular